# Quora Question pairs


Today it's a preparation for delivery 1.


The objective of today is to build a solution for:

https://www.kaggle.com/c/quora-question-pairs/overview

You might even make a late submission to kaggle to know how well you would have done in the competition.

##### DEADLINE: March 31


Tasks for the delivery:

- Make a solution for this problem using a CountVectorizer and TfidfVectorizer implemented by yourself.

- Benchmark your CountVectorizer and TfidfVectorizer with respect to the ones found in sklearn. You should also look if the output is the same.

- Show train validation results.

- Select model using validation (or crossvalidation) metrics. Using this model get the accuracy in the test set. 

In [150]:
import pandas as pd
import scipy
import sklearn
from sklearn import *
import numpy as np

In [80]:
cl = sklearn.linear_model.Perceptron()

In [81]:
# use this to train and VALIDATE your solutio
train_df = pd.read_csv("../data/quora_train_data.csv")

# use this to provide the expected generalization results
# test_df = pd.read_csv("../data/quora_test_data.csv")

In [82]:
# train_df

In [83]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323432 entries, 0 to 323431
Data columns (total 6 columns):
id              323432 non-null int64
qid1            323432 non-null int64
qid2            323432 non-null int64
question1       323431 non-null object
question2       323430 non-null object
is_duplicate    323432 non-null int64
dtypes: int64(4), object(2)
memory usage: 14.8+ MB


In [84]:
train_df, test_df = sklearn.model_selection.train_test_split(train_df, 
                                                             test_size=0.1, 
                                                             random_state=123)

In [85]:
train_df.shape, test_df.shape

((291088, 6), (32344, 6))

Let us train a first model....

In [86]:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))

all_q1 = list(train_df["question1"]) + list(test_df["question1"])
all_q2 = list(train_df["question2"]) + list(test_df["question2"])
all_questions = all_q1 + all_q2

len(all_questions)

646864

ups! the fit method is not working

The following code will not work

```python
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))
count_vectorizer.fit(all_questions)
```

In [87]:
#count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))
#count_vectorizer.fit(all_questions)

We got an error what should we do?

```
ValueError: np.nan is an invalid document, expected byte or unicode string.
```


#####  Exercise 1:  `cast_list_as_strings`

Build a function  **`cast_list_as_strings`** that casts each element in the input list to a string.


In [88]:
import numpy as np
types_ = [type(q).__name__ for q in all_q1]
np.unique(types_)

array(['float', 'str'], dtype='<U5')

In [89]:
def cast_list_as_strings(mylist):
    """
    return a list of strings
    """
    #assert isinstance(mylist, list), f"the input mylist should be a list it is {type(mylist)}"
    mylist_of_strings = []
    for x in mylist:
        mylist_of_strings.append(str(x))

    return mylist_of_strings

In [90]:
q1_train =  cast_list_as_strings(list(train_df["question1"]))
q2_train =  cast_list_as_strings(list(train_df["question2"]))
q1_test  =  cast_list_as_strings(list(test_df["question1"]))
q2_test  =  cast_list_as_strings(list(test_df["question2"]))

In [91]:
import numpy as np
types_ = [type(q).__name__ for q in q2_train]
np.unique(types_)

array(['str'], dtype='<U3')


Use all the questions in train and test partitions to vuild a single list `all_questions` to fit the `count_vectorizer`

In [92]:
all_questions = q1_train + q2_train

In [152]:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))
count_vectorizer.fit(all_questions)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

#####  Exercise 2:  `get_features_from_df`

Make a function `get_features_from_df` that given a dataframe containing the format of the train data
it returns a scipy sparse matrix with the features from question 1 and question 2 

In [149]:

def get_features_from_df(df, count_vectorizer):
    """
    returns a sparse matrix containing the features build by the count vectorizer.
    Each row should contain features from question1 and question2.
    """
    q1_casted =  cast_list_as_strings(list(df["question1"]))
    q2_casted =  cast_list_as_strings(list(df["question2"]))
    
    ############### Begin exercise ###################
    # don't do a for loop
    #for sentencen in df.shape[0]:
    #    do here encoding
    X_q1 = count_vectorizer.transform(q1_casted)
    X_q2 = count_vectorizer.transform(q2_casted)
    X_q1q2 = scipy.sparse.hstack((X_q1, X_q2))    
    ############### End exercise ###################

    return X_q1q2

In [95]:
X_tr_q1q2 = get_features_from_df(train_df,count_vectorizer)
X_te_q1q2  = get_features_from_df(test_df, count_vectorizer)

X_tr_q1q2.shape, train_df.shape, test_df.shape, X_te_q1q2.shape

((291088, 149518), (291088, 6), (32344, 6), (32344, 149518))

Now You can use this representation `X_tr_q1q2` to fit a model

In [153]:
logistic = sklearn.linear_model.LogisticRegression(solver="liblinear",
                                                   random_state=123)
y_train = train_df["is_duplicate"].values
logistic.fit(X_tr_q1q2, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=123, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

## Examine mistakes


#####  Exercise 3:  `get_mistakes`

Make a function `get_mistakes` that given a model `clf` a dataframe `df`, the features `X_q1q2` and the target labels `y`returns 


- incorrect_indices: coordinates where the model made a mistake
- predictions: predictions made by the model


In [154]:
def get_mistakes(clf, X_q1q2, y):

    ############### Begin exercise ###################
    predictions        = clf.predict(X_q1q2)
    incorrect_preds    = predictions != y
    incorrect_indices, = np.where(incorrect_preds)
    incorrect_indices2 = [x for x in  range(len(incorrect_preds)) if incorrect_preds[x] ==True]
    incorrect_indices3 = np.arange(len(incorrect_preds))[incorrect_preds]        
    ############### End exercise ###################
    
    if np.sum(incorrect_preds)==0:
        print("no mistakes in this df")
    else:
        return incorrect_indices, predictions

In [162]:
y_train = train_df["is_duplicate"].values

mistake_indices, predictions = get_mistakes(logistic,
                                            X_tr_q1q2, 
                                            y_train)

In [163]:
def print_mistake_k(k, mistake_indices, predictions):
    print(train_df.iloc[mistake_indices[k]].question1)
    print(train_df.iloc[mistake_indices[k]].question2)
    print("true class:", train_df.iloc[mistake_indices[k]].is_duplicate)
    print("prediction:", predictions[mistake_indices[k]])

In [174]:
#[x for x in X if "'s" in x]
print_mistake_k(10, mistake_indices, predictions)

Why do men like women's feet?
Why do men like womens feet?
true class: 1
prediction: 0


In [176]:
print_mistake_k(12, mistake_indices, predictions)

What are abiotic factors and how do they affect the ecosystem?
How do biotic and abiotic factors work together to make an ecosystem?
true class: 1
prediction: 0


# Improving feature vectors:

Build a Similarity metric between documents using the tf-idf vectors of the documents.

Build a class from scratch that can learn a tf-idf feature representation.